<a href="https://colab.research.google.com/github/roholy/Covid19/blob/main/covid_19_in_usa_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize delevopment of Corona pandemic in the USA

In [1]:
# inits
library(tidyverse)
library(lubridate) # for date handling

list.files(path = '../input/novel-corona-virus-2019-dataset/')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




character(0)

In [2]:
# load main file
df <- read.csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
str(df)

Warning message in file(file, "rt"):
“cannot open file '../input/novel-corona-virus-2019-dataset/covid_19_data.csv': No such file or directory”


ERROR: ignored

In [ ]:
# convert dates
df$Date <- as.Date(df$ObservationDate, format = '%m/%d/%y')
df$ObservationDate <- NULL
df$Last.Update <- NULL

In [ ]:
# filter US
df_USA <- dplyr::filter(df, Country.Region=='US')

In [ ]:
# aggregate states to nationwide view
df_USA <- dplyr::group_by(df_USA, Date) %>% summarise(Confirmed = sum(Confirmed),
                                                   Deaths = sum(Deaths),
                                                   Recovered = sum(Recovered))

In [ ]:
n <- nrow(df_USA)
print(paste0('Observations: ', n))
# add a few more features
df_USA$Active <- df_USA$Confirmed - df_USA$Recovered - df_USA$Deaths 
df_USA$MortRate <- df_USA$Deaths / df_USA$Confirmed
df_USA$RecRate <- df_USA$Recovered / df_USA$Confirmed
df_USA$Rec_vs_Mort <- df_USA$Recovered / df_USA$Deaths
df_USA$DiffConf <- 0
df_USA$DiffConf[2:nrow(df_USA)] <- diff(df_USA$Confirmed)
df_USA$RelIncrease <- c(NaN, df_USA$DiffConf[2:n] / df_USA$Confirmed[1:n-1])
df_USA$DoublingDays <- round(log(2)/log(1+df_USA$RelIncrease),2)
df_USA$DiffDeath <- 0
df_USA$DiffDeath[2:nrow(df_USA)] <- diff(df_USA$Deaths)
df_USA$RelIncD <- c(NaN, df_USA$DiffDeath[2:n] / df_USA$Deaths[1:n-1])
df_USA$DoublingDaysD <- round(log(2)/log(1+df_USA$RelIncD),2)
# show data frame
df_USA

In [ ]:
print(paste0('Data as at: ', max(df_USA$Date)))

In [ ]:
plot(df_USA$Date, df_USA$Confirmed, main='Confirmed Cases', type='b')
grid()

In [ ]:
plot(df_USA$Date, log10(df_USA$Confirmed), main='Confirmed Cases - Log Scale', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$DiffConf, main='Absolute Increase of Confirmed Cases', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$RelIncrease, main='Relative Change vs Previous Day', type='b', ylim=c(0,0.5))
grid()

In [ ]:
plot(df_USA$Date, df_USA$DoublingDays, main='Confirmed - Theoretical days until next doubling', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$Deaths, main='Deaths', type='b')
grid()

In [ ]:
plot(df_USA$Date, log10(df_USA$Deaths), main='Deaths - Log Scale', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$DiffDeath, main='Absolute Increase of Deaths', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$RelIncD, main='Deaths - Relative Change vs Previous Day', type='b', ylim=c(0,0.5))
grid()

In [ ]:
plot(df_USA$Date, df_USA$DoublingDaysD, main='Deaths - Theoretical days until next doubling', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$Recovered, main='Recovered', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$Active, main='Active', type='b')
grid()

In [ ]:
plot(df_USA$Date, df_USA$RecRate, pch=16, col='darkgreen', type='b', ylab='Recovery Ratio', main='Recovery Ratio')
grid()

In [ ]:
plot(df_USA$Date, df_USA$MortRate, pch=16, col='red', type='b', ylab='Death Ratio', main='Death Ratio')
grid()

In [ ]:
plot(df_USA$Date, df_USA$Rec_vs_Mort, pch=16, col='blue', type='b', ylab='Recovered : Deaths', main='Recovered : Deaths')
grid()

In [ ]:
# bubble plot (death ratio, recovery ratio, total positive cases)
ggplot(df_USA, aes(x=RecRate, y=MortRate, size=Confirmed, color=Date)) +
      geom_point(alpha=0.7) + ggtitle('Ratios') + theme(legend.position="bottom")

In [ ]:
# store in CSV for further processing
write.csv(df_USA, file='USA.csv')

### See also the following notebooks:
* Italy: https://www.kaggle.com/docxian/corona-italia-region-province
* Spain: https://www.kaggle.com/docxian/corona-in-spain-r
* Germany: https://www.kaggle.com/docxian/corona-in-germany-r